In [1]:
import glob 
import os
import pickle
import numpy as np
import torch
from torch.nn import functional as F
import torch.optim as optim
from tqdm import tqdm
import matplotlib.pyplot as plt

from torch.distributions import Laplace
import poisevae
from poisevae.datasets import CUB
from poisevae.utils import NN_lookup, Categorical, sent_emb
from poisevae.networks.CUBNetworks import EncImg, DecImg, EncTxt, DecTxt

from sklearn.cross_decomposition import CCA

# from cca import *

plt.rcParams['pdf.fonttype'] = 42
plt.rcParams['ps.fonttype'] = 42
plt.rcParams['font.family'] = 'Times New Roman'
plt.rcParams['font.size'] = 20
plt.rcParams['font.weight'] = 'normal'
plt.rcParams['mathtext.fontset'] = 'cm'
plt.rcParams['text.usetex'] = False

## Declarations & Loading

In [2]:
device ='cuda' if torch.cuda.is_available() else 'cpu'

In [3]:
HOME_PATH = os.path.expanduser('~')
DATA_PATH = os.path.join(HOME_PATH, 'Datasets/CUB/')

In [4]:
DATA_SIZE = 81920
true_img = torch.load('../../true_data_img.pt')[:DATA_SIZE]
true_txt = torch.load('../../true_data_txt.pt')[:DATA_SIZE]
true_img_pca = torch.load('../../true_data_img_pca.pt').cpu().numpy()[:DATA_SIZE]
true_sent_emb = torch.load('../../true_data_sent_embedding.pt').cpu().numpy()[:DATA_SIZE]
sent_PC = torch.load('../../sentence_emb_PC.pt').to(device, torch.float32)
img_PC = torch.load('../../image_PC.pt').to(device, torch.float32)

In [5]:
with open(os.path.join(DATA_PATH, 'cub/oc:3_msl:32/cub.emb'), 'rb') as file:
    emb = pickle.load(file)
with open(os.path.join(DATA_PATH, 'cub/oc:3_msl:32/cub.weights'), 'rb') as file:
    weights = pickle.load(file)

In [6]:
def pca_transform(X, PC):
    li = X.split(2048, 0)
    return torch.cat([e - torch.matmul(PC, e.unsqueeze(-1)).squeeze() for e in li])

In [7]:
def calculate_corr(imgs, txts, true_img_mean, true_txt_mean):
    if isinstance(imgs, np.ndarray):
        imgs = torch.from_numpy(imgs)
    if isinstance(txts, np.ndarray):
        txts = torch.from_numpy(txts)
    if isinstance(true_img_mean, np.ndarray):
        true_img_mean = torch.from_numpy(true_img_mean)
    if isinstance(true_txt_mean, np.ndarray):
        true_txt_mean = torch.from_numpy(true_txt_mean)
    # Assume all are projected
    corr = F.cosine_similarity((imgs.cpu() - true_img_mean.cpu()), 
                               (txts.cpu() - true_txt_mean.cpu())).mean()
    return corr

In [8]:
def perform_cca(gen_img=None, gen_txt_emb=None):
    if gen_img is not None:
        gen_img = torch.cat(gen_img).to(device, torch.float32)
        gen_img = pca_transform(gen_img, img_PC.to(device, torch.float32)).cpu().numpy()
    else:
        gen_img = true_img_pca
        
    if gen_txt_emb is not None: 
        gen_txt_emb = torch.from_numpy(np.vstack(gen_txt_emb)).to(device, torch.float32)
        gen_txt_emb = pca_transform(gen_txt_emb, sent_PC.to(device, torch.float32)).cpu().numpy()
    else:
        gen_txt_emb = true_sent_emb
    
    gen_img_cca, gen_sent_cca = cca.transform(gen_img, gen_txt_emb)
    return float(calculate_corr(gen_img_cca, gen_sent_cca, true_img_cca_mean, true_sent_cca_mean))

def eval_model(vae, mode, condition):
    corr = []
    for path in glob.glob('../example/runs/CUB/%s/*' % condition):
        try:
            vae, _, _ = poisevae.utils.load_checkpoint(vae, load_path=os.path.join(path, 'training_40.pt'))
        except FileNotFoundError:
            continue
        vae = vae.to(device, torch.float32)
        gen_txt_emb = []
        gen_img = []
        with torch.no_grad():
            for i in range(len(true_img)):
                if mode == 'i2s':
                    results = vae([true_img[i], None])
                    gen_txt_i = results['x_rec'][1].probs.argmax(dim=1).reshape(true_img[i].shape[0], -1)
                    gen_txt_emb.append(np.zeros((true_img[i].shape[0], emb.shape[1])))
                    sent_emb(gen_txt_i.cpu().numpy().astype(np.int32), emb, weights, gen_txt_emb[-1])
                elif mode == 's2i':
                    results = vae([None, true_txt[i]])
                    gen_img.append(results['x_rec'][0].loc)
                elif mode == 'joint':
                    results = vae([None, None])
                    gen_img.append(results['x_rec'][0].loc)
                    gen_txt_i = results['x_rec'][1].probs.argmax(dim=1).reshape(true_img[i].shape[0], -1)
                    gen_txt_emb.append(np.zeros((true_img[i].shape[0], emb.shape[1])))
                    sent_emb(gen_txt_i.cpu().numpy().astype(np.int32), emb, weights, gen_txt_emb[-1])
                else: 
                    raise ValueError

            if mode == 'i2s':
                corr.append(perform_cca(gen_txt_emb=gen_txt_emb))
            elif mode == 's2i':
                corr.append(perform_cca(gen_img=gen_img))
            elif mode == 'joint':
                corr.append(perform_cca(gen_img=gen_img, gen_txt_emb=gen_txt_emb))
            else: 
                raise ValueError 
            
    return corr

## CCA over truths

In [9]:
try:
    with open('CCA_model.pkl', 'rb') as f:
        cca = pickle.load(f)
except FileNotFoundError:
    cca = CCA(n_components=10, tol=1e-4)
    cca.fit(true_img_pca, true_sent_emb)
    with open('CCA_model.pkl','wb') as f:
        pickle.dump(cca, f)

In [10]:
true_img_cca, true_sent_cca = cca.transform(true_img_pca, true_sent_emb)
true_img_cca_mean, true_sent_cca_mean =  true_img_cca.mean(axis=0), true_sent_cca.mean(axis=0)
calculate_corr(true_img_cca, true_sent_cca, true_img_cca_mean, true_sent_cca_mean)

tensor(0.4875, dtype=torch.float64)

### Check on convergence

In [11]:
idx = np.arange(true_img_pca.shape[0])
np.random.shuffle(idx)
true_img_cca_hat, true_sent_cca_hat = cca.transform(true_img_pca[idx], true_sent_emb[idx])
calculate_corr(true_img_cca_hat, true_sent_cca_hat, true_img_cca_mean, true_sent_cca_mean)

tensor(0.4875, dtype=torch.float64)

### Check on overfitting

In [12]:
# test_true_img_pca = torch.load('true_data_img_pca.pt').cpu().numpy()[DATA_SIZE:DATA_SIZE+1000]
# test_true_sent_emb = torch.load('true_data_sent_embedding.pt').cpu().numpy()[DATA_SIZE:DATA_SIZE+1000]
# test_true_img_cca_hat, test_true_sent_cca_hat = cca.transform(test_true_img_pca, test_true_sent_emb)
# calculate_corr(test_true_img_cca_hat, test_true_sent_cca_hat, true_img_cca_mean, true_sent_cca_mean)

## CCA over trained model

### Loading checkpoints

In [13]:
condition = 'worew' # without reweighting
enc_img = EncImg(128).to(device, torch.float32)
dec_img = DecImg(128).to(device, torch.float32)
enc_txt = EncTxt(1590, 128).to(device, torch.float32)
dec_txt = DecTxt(1590, 128).to(device, torch.float32)
vae = poisevae.POISEVAE([enc_img, enc_txt], [dec_img, dec_txt], likelihoods=[Laplace, Categorical], 
                        latent_dims=[128, (128, 1, 1)], device=device)

In [14]:
true_img = true_img.to(device, torch.float32).split(2048, dim=0)
true_txt = true_txt.to(device, torch.float32).split(2048, dim=0)

### Cross Generation

#### Image -> Caption

In [15]:
corr = eval_model(vae, 'i2s', condition)
corr

[0.3873777559642452, 0.3476689747878726, 0.3324589801101878]

#### Image <- Caption

In [18]:
corr = eval_model(vae, 's2i', condition)
corr

[0.012597809681969089, -0.006929092858105511, 0.032757421965073676]

### Joint Generation

In [19]:
corr = eval_model(vae, 'joint', condition)
corr

[0.5120772018741817, 0.3761393199694043, 0.31803801406889864]